In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import metrics

Here we are attempting to use a support vector machine classifier to help indicate if a municipality should be on the UAL.
We will first attempt to predict if they areon the General Fund UAL (with and without water data)

In [29]:
data = pd.read_excel('./Data/UAL_2015_to_2018_V2.xlsx')
test = data.loc[data['Audit Year'] == 2018].drop(['Audit Year'], axis = 1)
train = data.loc[data['Audit Year'] != 2018].drop(['Audit Year'], axis = 1)
out = test.loc[:,['UnitYear', 'Internal Controls', 'Financial Issues General Fund', 'Financial Issues Water Sewer Fund']]
names = test['UnitYear'].values

Again we will start with the general fund, so let's drop the other indicating columns.

In [3]:
print(test.columns)

Index(['UnitYear', 'FBA', 'FBA w/o Powell Bill', 'Total Expenditures',
       '<8% FBA/Expenditures', '<8% FBA w/o Powell/ Expenditures',
       'WS Quick Ratio <1', 'WS Working Capital',
       'WS Cash Flow from ops less debt service', 'WS-EF- Interest Expense',
       'Internal Controls', 'Financial Issues General Fund',
       'Financial Issues Water Sewer Fund'],
      dtype='object')


In [30]:
test = test.drop(['Financial Issues Water Sewer Fund', 'Internal Controls'], axis = 1)
train = train.drop(['Financial Issues Water Sewer Fund', 'Internal Controls'], axis = 1)


now we will perform the SVM and test its accuracy.

In [31]:
C = 1.0
features = test.columns[1:-1].values
print(features)
Y = train['Financial Issues General Fund']
X = train[features]
svc = svm.SVC(kernel='rbf', C=C).fit(X, Y)

['FBA' 'FBA w/o Powell Bill' 'Total Expenditures' '<8% FBA/Expenditures'
 '<8% FBA w/o Powell/ Expenditures' 'WS Quick Ratio <1'
 'WS Working Capital' 'WS Cash Flow from ops less debt service'
 'WS-EF- Interest Expense']


C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [32]:
x_test = test[features]
y_test = test['Financial Issues General Fund']
on_ual = svc.predict(x_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual))


Accuracy :  0.9248747913188647


In [7]:
add = pd.DataFrame({'UnitYear':names, 'Genral Fund Prediction': on_ual})
out = pd.merge(out, add, how = 'left', on = 'UnitYear')

Now Lets try without the water sewer data.

In [8]:
C = 1.0
features = test.columns[1:6].values
print(features)
X = train[features]
svc = svm.SVC(kernel='rbf', C=C).fit(X, Y)

['FBA' 'FBA w/o Powell Bill' 'Total Expenditures' '<8% FBA/Expenditures'
 '<8% FBA w/o Powell/ Expenditures']


C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [33]:
x_test = test[features]
on_ual = svc.predict(x_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual))

Accuracy :  0.9248747913188647


In [11]:
sum(on_ual)

0

It appears that the statistuically best way to approach this is to say that NO ONE is on the UAL. This is a fairly subvjective list, and the vast majority are not in the UAL, so this is not too surprising.

This is data that has not been normalized. We will do that now and redo the model.

In [13]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

In [24]:
scaler = StandardScaler().fit(X)  
scaled_X = scalx = scaler.transform(X)

normer = Normalizer().fit(X)
norm_X = normer.transform(X)

svc_S = svm.SVC(kernel='rbf', C=C).fit(scaled_X, Y)
svc_N = svm.SVC(kernel='linear', C=C).fit(norm_X, Y)

C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [25]:
scaler = StandardScaler().fit(x_test)  
scaled_X_test = scalx = scaler.transform(x_test)

normer = Normalizer().fit(x_test)
norm_X_test = normer.transform(x_test)

C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [26]:
on_ual_S = svc_S.predict(scaled_X_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual_S))

on_ual_N = svc_N.predict(norm_X_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual_N))

Accuracy :  0.8797996661101837
Accuracy :  0.8797996661101837


Again, we see the same phenomenon. Perhaps SVM is not the way to go here. There is likely not a large enough disparity in the vector space between on the UAL and not on the UAL.

Now lets try predicting something that isn't so straightforward and has a fair degree of subjectivity.
Internal Control Issues

In [17]:
test = data.loc[data['Audit Year'] == 2018].drop(['Audit Year'], axis = 1)
train = data.loc[data['Audit Year'] != 2018].drop(['Audit Year'], axis = 1)

test = test.drop(['Financial Issues Water Sewer Fund', 'Financial Issues General Fund'], axis = 1)
train = train.drop(['Financial Issues Water Sewer Fund', 'Financial Issues General Fund'], axis = 1)

y_test = test['Internal Controls']
Y = train['Internal Controls']

In [21]:
svc_S = svm.SVC(kernel='linear', C=C).fit(scaled_X, Y)
svc_N = svm.SVC(kernel='linear', C=C).fit(norm_X, Y)

In [23]:
on_ual_S = svc_S.predict(scaled_X_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual_S))

on_ual_N = svc_N.predict(norm_X_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual_N))

print(sum(on_ual_N))

Accuracy :  0.8797996661101837
Accuracy :  0.8797996661101837
0


In [148]:
add = pd.DataFrame({'UnitYear':names, 'Internal Control Prediction': on_ual_S})
out = pd.merge(out, add, how = 'left', on = 'UnitYear')

Great! Not Bad for something that is fairly subjective. Now lets try water sewer.

First let's re-introduce the water sewer data, and drop all units that don't actually run a water utility indicated by an 'n' in the rating column.

In [149]:
test = data.loc[data['Audit Year'] == 2018].drop(['Audit Year'], axis = 1).loc[data['Financial Issues Water Sewer Fund'] != -1]
train = data.loc[data['Audit Year'] != 2018].drop(['Audit Year'], axis = 1).loc[data['Financial Issues Water Sewer Fund'] != -1]


test = test.drop(['Financial Issues General Fund', 'Internal Controls'], axis = 1)
train = train.drop(['Financial Issues General Fund', 'Internal Controls'], axis = 1)

In [150]:
C = 1.0
features = test.columns[6:-1].values
print(features)

Y = train['Financial Issues Water Sewer Fund']
X = train[features]

svc = svm.SVC(kernel='rbf', C=C).fit(X, Y)

['WS Quick Ratio <1' 'WS Working Capital'
 'WS Cash Flow from ops less debt service' 'WS-EF- Interest Expense']


C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [151]:
x_test = test[features]
y_test = test['Financial Issues Water Sewer Fund']
on_ual = svc.predict(x_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual))

Accuracy :  0.8411910669975186


In [152]:
names = test['UnitYear'].values
add = pd.DataFrame({'UnitYear':names, 'Water Sewer Fund Prediction': on_ual})
out = pd.merge(out, add, how = 'left', on = 'UnitYear')

In [153]:
scaler = StandardScaler().fit(X)  
scaled_X = scalx = scaler.transform(X)

scaler = StandardScaler().fit(x_test)  
scaled_X_test = scalx = scaler.transform(x_test)

svc_S = svm.SVC(kernel='rbf', C=C).fit(scaled_X, Y)

C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\lgc0069\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [154]:
on_ual_S = svc_S.predict(scaled_X_test)
print('Accuracy : ',metrics.accuracy_score(y_test, on_ual_S))

Accuracy :  0.8411910669975186


In [156]:
out.to_excel('./Data/UAL_Predictions.xlsx')